In [7]:
# disable compiler warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
(train_ds_global, test_ds_global), ds_info = tfds.load("cifar10" , split =["train", "test"] , as_supervised=True , with_info=True)

def prepare_cifar10_data(cifar10):
  #convert data from uint8 to float32
  cifar10 = cifar10.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  cifar10 = cifar10.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  cifar10 = cifar10.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  cifar10 = cifar10.cache()
  #shuffle, batch, prefetch
  cifar10 = cifar10.shuffle(1000)
  cifar10 = cifar10.batch(32)
  cifar10 = cifar10.prefetch(20)
  #return preprocessed dataset
  return cifar10

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Can-Leon\tensorflow_datasets\cifar10\3.0.2.incompleteN9UGUB\cifar10-train.tfrecord*...:   0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Can-Leon\tensorflow_datasets\cifar10\3.0.2.incompleteN9UGUB\cifar10-test.tfrecord*...:   0%…

Dataset cifar10 downloaded and prepared to C:\Users\Can-Leon\tensorflow_datasets\cifar10\3.0.2. Subsequent calls will reuse this data.


In [9]:
class BasicConv(tf.keras.Model):
    def __init__(self):
        super(BasicConv, self).__init__()

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.metrics_list = [tf.keras.metrics.Accuracy(name="Accuracy"),
                             tf.keras.metrics.Mean(name="loss")]

        self.optimizer = tf.keras.optimizers.Adam()

        self.convlayer1 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu')
        self.convlayer2 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu')
        self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

        self.convlayer3 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
        self.convlayer4 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
        self.global_pool = tf.keras.layers.GlobalAvgPool2D()

        self.out = tf.keras.layers.Dense(10, activation='softmax')

    @tf.function
    def call(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = self.pooling(x)
        x = self.convlayer3(x)
        x = self.convlayer4(x)
        x = self.global_pool(x)
        x = self.out(x)
        return x

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model



    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()



    # train_step method
    @tf.function
    def train_step(self, data):
        img, label = data
        
        # compute output and loss, train the variables
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            loss = self.loss_function(label, output)
            
        # update trainable variables
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update metrics
        self.metrics_list[0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1].update_state(loss)
        
        # return a dict with metric information
        return {m.name : m.result() for m in self.metrics_list}



    # test_step method
    @tf.function
    def test_step(self, data):
        img, label = data

        # compute output and loss, without training
        output = self(img, training=False)
        loss = self.loss_function(label, output)

        # update metrics
        self.metrics_list[0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1].update_state(loss)

        # return a dict with metric information 
        return {m.name : m.result() for m in self.metrics_list}

In [10]:

def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

train_summary_writer, val_summary_writer = create_summary_writers(config_name="RUN1")

In [11]:
# trains the model by iterating through the dataset and applying training_step method epochs time
def training_loop(model, train_ds, epochs, train_summary_writer):
    # iterate over epochs
    for epoch in range(epochs):
        print(f"epoch: {epoch}")

        # train steps on all batches in the training data
        for (img, label) in train_ds:
            metrics = model.train_step((img, label))
            
            # keep data in summary with metrics
            with train_summary_writer.as_default():
                for metric in model.metrics_list:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
        
        # print current metric values and reset the metrics
        print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
        model.reset_metrics()


# tests overall performance of model
def test_loop(model, test_ds, val_summary_writer):
    # test steps on every item in test dataset
    for (img, label) in test_ds:
        metrics = model.test_step((img, label))
        
        # keep data with metrics
        with val_summary_writer.as_default():
            for metric in model.metrics_list:
                tf.summary.scalar(f"{metric.name}", metric.result(), step=1)
    
    #print current metric values and reset the metrics
    print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
    model.reset_metrics()

In [12]:
"""def try_model(model, ds):
  for x, t in ds.take(5):
    y = model(x)
    print(tf.argmax(y, axis=1), tf.argmax(t, axis=1))

basic_model = BasicConv()
try_model(basic_model, train_dataset)"""

train_dataset = train_ds_global.apply(prepare_cifar10_data)
test_dataset = test_ds_global.apply(prepare_cifar10_data)

print(test_dataset)

basic_model = BasicConv()

# about 10% accuracy, makes sense, since output is always the same and there are 10 labels
test_loop(basic_model,
        test_ds=test_dataset,
        val_summary_writer=val_summary_writer)

training_loop(basic_model,
              train_ds=train_dataset,
              epochs=10, 
              train_summary_writer=train_summary_writer)

test_loop(basic_model,
        test_ds=test_dataset,
        val_summary_writer=val_summary_writer)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>
['Accuracy : 0.10019999742507935', 'loss : 2.3041656017303467']
epoch: 0
['Accuracy : 0.3616800010204315', 'loss : 1.7040834426879883']
epoch: 1
['Accuracy : 0.5161799788475037', 'loss : 1.3529080152511597']
epoch: 2
['Accuracy : 0.577019989490509', 'loss : 1.190583348274231']
epoch: 3
['Accuracy : 0.6071599721908569', 'loss : 1.1034165620803833']
epoch: 4
['Accuracy : 0.6338599920272827', 'loss : 1.0341838598251343']
epoch: 5
['Accuracy : 0.6509400010108948', 'loss : 0.9830000996589661']
epoch: 6
['Accuracy : 0.6686599850654602', 'loss : 0.9371224045753479']
epoch: 7
['Accuracy : 0.6844800114631653', 'loss : 0.897203803062439']
epoch: 8
['Accuracy : 0.6960600018501282', 'loss : 0.8650387525558472']
epoch: 9
['Accuracy : 0.7092800140380859', 'loss : 0.832627534866333']
['Accuracy : 0.6984000205993652', 'loss : 0.8532191514968872']


In [14]:
%tensorboard --logdir logs

Launching TensorBoard...